<a href="https://colab.research.google.com/github/LouisCMS/Eki.thons/blob/master/remote_ekiHackhaton-LoC-v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Introduction
Welcome to the first remote Eki hackaton. The aim of the hackaton is to build your first neural network and a learning scheme in a efficient way.The task is image classification on CIFAR10

**************
The code contains :

1. Load and normalizing the CIFAR10 training and test datasets using
   ``torchvision``
2. Define a Convolutional Neural Network
3. Define a loss function
4. Train the network on the training data
5. Test the network on the test data

**************
Some **questions** too keep in mind :

    - which kind of data transformation and augmentation?
    - How building my neural network?
    - which kind of optimization?

Some tips here :) : [ML Lunch](https://bizoffice61.sharepoint.com/:p:/r/sites/Ekimetrics51/_layouts/15/Doc.aspx?sourcedoc=%7BA44BD8EB-39E2-4113-9861-A817680A7F55%7D&file=20200304%20-%20ML%20Lunch%20-%20Understanding%20Deep%20Learning.pptx&action=edit&mobileredirect=true)

**************
How to win?

The winner will propose a script which:

reachs at least 60% of accuracy on the dataset
builds a network with a minimum number of parameters.
Example :

People 1 : Accuracy : 58%, number of parameter : 10K

People 2 : Accuracy : 61%, number of parameters : 15K

People 3: Accuracy : 67%, number of paremeters : 16K

People 2 wins !

#Import all the libraries

In [0]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import requests

# Put the token you received in a mail here ! 
token = ''



---



# Load and normalizing the CIFAR10 training and test datasets using ``torchvision``

In [0]:
########################################################################
# The output of torchvision datasets are PILImage images of range [0, 1].
# We transform them to Tensors of normalized range [-1, 1].

transform = transforms.Compose(
     [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=0)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=0)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')



Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


#Define a Convolutional Neural Network 


In [0]:
class Net(nn.Module):
    def __init__(self):
        """
        This is your network. Here you will define all the layers of the network.
        In this example : conv1: first convolutional layer.
                        pool : the pooling function you want to use (you can use several pooling functions)
                        fc1 : the first fully connected layer.
        """
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(1176, 10)

    def forward(self, x):
        '''
        make the forward computation 
        x : the normalised data (this is a batch of images, ie a 4D tensor)
        '''
        # first layer : convolution then relu then pooling function
        x = self.pool(F.relu(self.conv1(x)))
        # flatten the matrix before passing into the first fc layer
        x = x.view(-1, 6 * 14 * 14)
        # second layer : fc then relu
        x = F.relu(self.fc1(x))
        return x

net = Net()

def count_parameters(model):
  '''
  counts the number of parameters in yout model. Please do not touch to this function
  '''
  return sum(p.numel() for p in model.parameters() if p.requires_grad)


# Define a Loss function and optimizer

In [0]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.1)

#Train the network

In [0]:
nepochs = 1

for epoch in range(nepochs):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

#Test the network on the test data : DO NOT TOUCH THIS PART

---

Your trained network is tested and the result is sent to the eki platform : 
http://eki-hackathon.herokuapp.com/



In [0]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
trainable_parameters = count_parameters(net)
print('Accuracy of the network on the 10000 test images: %d %%' % (
    accuracy))

print("number of trainable parameters : %d"%trainable_parameters)

score = accuracy if accuracy <= 60. else 60. + 40 * np.tanh(10000/trainable_parameters)
print(score)

requests.put(
  'http://eki-hackathon.herokuapp.com/submitScoreAndMetrics', 
  json={
    'id': token,
    'score': score,
    'metrics': {"trainable parameters": trainable_parameters,
                "accuracy" : accuracy}
  })

